<a href="https://colab.research.google.com/github/jiaweitang22/transformers/blob/main/Copy_of_ditto_light.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install packages

In [ ]:
# Execute the cell only once if you do not have Ditto code and required environment. 
# After the execution, you have three files: ditto, apex and sample_data

!git clone https://github.com/megagonlabs/ditto
%cd ditto/
!pip install -r requirements.txt
!pip install transformers
!pip install tensorboardX
!pip install jsonlines
%cd ..
import nltk
nltk.download('stopwords')
!git clone https://github.com/NVIDIA/apex
%cd apex
!pip install -v --no-cache-dir ./
%cd ..
!pip install --upgrade "urllib3==1.25.4" awscli # some issue with colab

Cloning into 'ditto'...
remote: Enumerating objects: 291, done.
remote: Total 291 (delta 0), reused 0 (delta 0), pack-reused 291
Receiving objects: 100% (291/291), 26.87 MiB | 17.57 MiB/s, done.
Resolving deltas: 100% (142/142), done.
Checking out files: 100% (128/128), done.
/content/ditto
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.2 MB 3.3 MB/s 
     |████████████████████████████████| 14.5 MB 44.2 MB/s 
     |████████████████████████████████| 689 kB 64.1 MB/s 
     |████████████████████████████████| 25.2 MB 3.7 MB/s 
     |████████████████████████████████| 1.0 MB 59.1 MB/s 
     |████████████████████████████████| 6.4 MB 31.6 MB/s 
ERROR: Could not find a version that satisfies the requirement torch==1.9.0+cu111 (from versions: 1.0.0, 1.0.1, 1.0.1.post2, 1.1.0, 1.2.0, 1.3.0, 1.3.1, 1.4.0, 1.5.0, 1.5.1, 1.6.0, 1.7.0, 1.7.1, 1.8.0, 1.8.1, 1.9.0, 1.9.1, 1.10.0, 1.10.1, 1.10.2, 1.11.0, 1.12.

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Cloning into 'apex'...
remote: Enumerating objects: 9873, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 9873 (delta 0), reused 1 (delta 0), pack-reused 9872
Receiving objects: 100% (9873/9873), 14.83 MiB | 10.81 MiB/s, done.
Resolving deltas: 100% (6794/6794), done.
/content/apex
Using pip 21.1.3 from /usr/local/lib/python3.7/dist-packages/pip (python 3.7)
Value for scheme.platlib does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
Value for scheme.purelib does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
Value for scheme.headers does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/include/python3.7/UNKNOWN
sysconfig: /usr/include/python3.7m/UNKNOWN
Value for scheme.scripts does

In [72]:
from google.colab import drive
import pandas as pd
import numpy as np

drive.mount('/content/gdrive')
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# enter directory ditto 
!ls
%cd ditto

apex  ditto  gdrive  sample_data
/content/ditto


## Tokenization

In [ ]:
bike_df = pd.read_csv('/content/ditto/data/er_magellan/Structured/Bike/bike_labeled.csv')
bike_df.head(5)

,ltable.bike_name,ltable.city_posted,ltable.km_driven,ltable.price,ltable.color,rtable.bike_name,rtable.city_posted,rtable.km_driven,rtable.price,rtable.color,gold
0,Hero Honda CBZ Disc,Delhi,58000,15000,black,Hero Honda Passion Standard,Delhi,59000,11000,red,0
1,Hero Honda Glamour,Delhi,45000,23000,black,Hero Honda Hunk Standard,Delhi,44000,40000,red,0
2,Hero Honda Splendor Plus,Delhi,7350,38000,black,Hero Splendor Plus Alloy,Delhi,8000,38000,black,1
3,Yamaha Fazer Sport,Delhi,12000,46000,red,Yamaha FZ S Standard,Delhi,12000,49000,red,0
4,Yamaha YZF R15 Version 2.0,Delhi,8000,90000,red,Yamaha YZF R15 V 2.0,Delhi,9000,80000,red,1


In [ ]:
def tokenize(df, middle):
  content = ''
  # clean up names
  placeholder = bike_df.columns
  col_names = [s[7:] for s in placeholder]
  col_names[-1] = placeholder[-1]

  for row_index in range(len(df)):
    row = df.iloc[row_index]
    for i in np.arange(len(row)-1):
      content = content + 'COL' + ' ' + str(col_names[i]) + ' ' + 'VAL' + ' ' + str(row[i]) + ' '
      if i == middle:
        content = content + '% ' + '\t'
      elif i == len(row)-2:
        content = content + '% ' +'\t' + str(row[-1])
    content = content + '\n'
  return content

In [ ]:
def split_df(df, train_split, val_split, test_split):
  df = df.sample(n=len(df), replace=False)
  train_df = df[:int(len(df)*train_split)]
  val_df = df[int(len(df)*train_split):int(len(df)-len(df)*test_split)]
  test_df = df[int(len(df)-len(df)*test_split):]
  return [train_df, val_df, test_df]

In [ ]:
train = tokenize(split_df(bike_df, 0.6, 0.2, 0.2)[0], middle=4)
val = tokenize(split_df(bike_df, 0.6, 0.2, 0.2)[1], middle=4)
test = tokenize(split_df(bike_df, 0.6, 0.2, 0.2)[2], middle=4)

In [ ]:
file = open("/content/ditto/data/er_magellan/Structured/Bike"+'/train'+'.txt', "w")

# Saving the results in a text file
content = train
file.write(content)
file.close()

In [ ]:
file = open("/content/ditto/data/er_magellan/Structured/Bike"+'/valid'+'.txt', "w")

# Saving the results in a text file
content = val
file.write(content)
file.close()

In [ ]:
file = open("/content/ditto/data/er_magellan/Structured/Bike"+'/test'+'.txt', "w")

# Saving the results in a text file
content = test
file.write(content)
file.close()

## Running the matcher

In [104]:

!CUDA_VISIBLE_DEVICES=0 python train_ditto.py \
  --task Structured/Bike \
  --batch_size 32 \
  --max_len 64 \
  --lr 3e-5 \
  --n_epochs 20 \
  --finetuning \
  --lm bert \
  --fp16 \
  --save_model \
  --da drop_col 

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/transformers/utils/hub.py", line 694, in get_file_from_repo
    use_auth_token=use_auth_token,
  File "/usr/local/lib/python3.7/dist-packages/transformers/utils/hub.py", line 292, in cached_path
    local_files_only=local_files_only,
  File "/usr/local/lib/python3.7/dist-packages/transformers/utils/hub.py", line 495, in get_from_cache
    _raise_for_status(r)
  File "/usr/local/lib/python3.7/dist-packages/transformers/utils/hub.py", line 418, in _raise_for_status
    f"401 Client Error: Repository not found for url: {response.url}. "
transformers.utils.hub.RepositoryNotFoundError: 401 Client Error: Repository not found for url: https://huggingface.co/bert/resolve/main/tokenizer_config.json. If the repo is private, make sure you are authenticated.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "train_ditto.py", line 84, in <module>
    da=hp.

In [105]:
!CUDA_VISIBLE_DEVICES=0 python matcher.py \
  --task Structured/Bike \
  --input_path data/er_magellan/Structured/Bike/test.txt \
  --output_path output/output_small.jsonl \
  --lm bert \
  --max_len 64 \
  --use_gpu \
  --fp16 \
  --checkpoint_path checkpoints/

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py", line 609, in _get_config_dict
    user_agent=user_agent,
  File "/usr/local/lib/python3.7/dist-packages/transformers/utils/hub.py", line 292, in cached_path
    local_files_only=local_files_only,
  File "/usr/local/lib/python3.7/dist-packages/transformers/utils/hub.py", line 495, in get_from_cache
    _raise_for_status(r)
  File "/usr/local/lib/python3.7/dist-packages/transformers/utils/hub.py", line 418, in _raise_for_status
    f"401 Client Error: Repository not found for url: {response.url}. "
transformers.utils.hub.RepositoryNotFoundError: 401 Client Error: Repository not found for url: https://huggingface.co/bert/resolve/main/config.json. If the repo is private, make sure you are authenticated.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "matcher.py", line 315, in <module>
    hp.lm, hp.use_gpu, 

### Accuracy

In [106]:
import jsonlines



with jsonlines.open('/content/ditto/output/output_small.jsonl') as f:
    bike_predictions = []
    for line in f.iter():
      # print(line['match']) # or whatever else you'd like to do
      bike_predictions = bike_predictions + [line['match']]
bike_predictions = np.array(bike_predictions)

In [107]:
bike_predictions

array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0,
       1, 1])

In [108]:
with open('/content/ditto/data/er_magellan/Structured/Bike/test.txt', "r") as f:
  data = f.readlines()
  bike_ground_truth = np.array([int(x[-2:-1]) for x in data])

In [109]:
bike_ground_truth

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 0])

In [110]:
bike_ground_truth == bike_predictions

array([ True,  True,  True,  True, False,  True,  True,  True,  True,
        True, False,  True, False,  True,  True, False,  True,  True,
        True, False, False,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True, False, False, False,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True, False, False,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True, False, False,  True,  True,  True,  True,  True,
       False,  True,  True,  True, False,  True,  True, False,  True,
        True,  True,  True,  True, False, False,  True,  True, False])

In [111]:
guesses = bike_predictions
labels = bike_ground_truth
r = recall_score(labels, guesses)
p = precision_score(labels, guesses)
f1 = f1_score(labels, guesses)
print('Recall:',r)
print('Precision:',p)
print('F1:',f1)

Recall: 0.7727272727272727
Precision: 0.5483870967741935
F1: 0.6415094339622641
